<a href="https://colab.research.google.com/github/ExplorerGumel/Landslide-Prevention-and-Innovation-Challenge/blob/main/LandSlide_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold
from sklearn.metrics import f1_score
np.random.seed(42)
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv(r'/content/Train.csv')
test  = pd.read_csv(r'/content/Test.csv')
ss = pd.read_csv(r'/content/SampleSubmission.csv')

In [2]:
train.head()

,Sample_ID,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,...,17_sdoif,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif,Label
0,1,130,129,127,126,123,126,125,124,122,...,1.281779,1.281743,1.281720,1.281684,1.281811,1.281788,1.281752,1.281729,1.281693,0
1,2,161,158,155,153,151,162,159,155,153,...,1.359639,1.359608,1.359587,1.359556,1.359683,1.359662,1.359631,1.359610,1.359579,1
2,3,149,151,154,156,158,154,157,158,160,...,1.365005,1.365025,1.365055,1.365075,1.364937,1.364967,1.364988,1.365018,1.365038,0
3,4,80,78,77,75,73,80,78,77,75,...,1.100708,1.100738,1.100759,1.100789,1.100630,1.100650,1.100680,1.100700,1.100731,0
4,5,117,115,114,112,110,115,113,111,110,...,1.284180,1.284130,1.284056,1.284006,1.284125,1.284050,1.284001,1.283926,1.283876,0


In [3]:
train.isnull().sum()

Sample_ID      0
1_elevation    0
2_elevation    0
3_elevation    0
4_elevation    0
              ..
22_sdoif       0
23_sdoif       0
24_sdoif       0
25_sdoif       0
Label          0
Length: 227, dtype: int64

In [7]:
train.columns

Index(['Sample_ID', '1_elevation', '2_elevation', '3_elevation', '4_elevation',
       '5_elevation', '6_elevation', '7_elevation', '8_elevation',
       '9_elevation',
       ...
       '17_sdoif', '18_sdoif', '19_sdoif', '20_sdoif', '21_sdoif', '22_sdoif',
       '23_sdoif', '24_sdoif', '25_sdoif', 'Label'],
      dtype='object', length=227)

This dataset have 227 columns which makes it heard to visualize once at a time using methods like .head, .info .describe and likes. So, to do that i view them based on their parent names  

In [14]:
elevation_cols = [col for col in train.columns if col.endswith('elevation')]
train[elevation_cols]

,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,10_elevation,...,16_elevation,17_elevation,18_elevation,19_elevation,20_elevation,21_elevation,22_elevation,23_elevation,24_elevation,25_elevation
0,130,129,127,126,123,126,125,124,122,119,...,119,117,115,114,112,116,114,113,111,110
1,161,158,155,153,151,162,159,155,153,151,...,162,160,157,153,151,162,160,157,154,150
2,149,151,154,156,158,154,157,158,160,161,...,166,168,168,167,167,170,171,171,170,169
3,80,78,77,75,73,80,78,77,75,73,...,80,78,77,75,73,79,78,76,73,72
4,117,115,114,112,110,115,113,111,110,108,...,110,109,107,105,104,108,106,105,103,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,203,202,201,200,199,202,201,201,199,198,...,198,198,196,195,194,196,195,194,193,192
10860,432,430,427,423,421,431,429,426,422,419,...,427,426,424,421,418,425,424,423,421,418
10861,348,345,342,339,335,344,342,339,336,332,...,338,336,332,329,326,336,333,330,326,323
10862,93,91,90,88,86,96,94,93,92,91,...,101,101,100,100,99,104,104,104,103,103


In [15]:
ls_cols = [col for col in train.columns if col.endswith('lsfactor')]
train[ls_cols]

,1_lsfactor,2_lsfactor,3_lsfactor,4_lsfactor,5_lsfactor,6_lsfactor,7_lsfactor,8_lsfactor,9_lsfactor,10_lsfactor,...,16_lsfactor,17_lsfactor,18_lsfactor,19_lsfactor,20_lsfactor,21_lsfactor,22_lsfactor,23_lsfactor,24_lsfactor,25_lsfactor
0,8.045186,8.333038,7.819405,8.032228,9.818933,9.295772,9.375107,9.499450,10.922910,11.001900,...,8.703197,10.175750,8.651121,8.251134,8.074524,8.519887,7.989215,7.634287,7.804186,7.219216
1,9.089893,8.944574,7.341060,5.833743,5.661154,7.023552,9.701378,9.165056,6.005569,5.987373,...,7.225154,7.111054,9.881402,9.140442,7.429777,7.356270,7.560902,9.259974,10.544930,9.779708
2,12.115220,13.973310,13.248880,11.468990,9.537852,13.846830,14.690480,15.173530,13.286840,11.810650,...,10.495960,8.481891,9.427755,10.075610,6.985573,8.660597,6.171284,6.751987,8.461001,9.387614
3,3.416162,3.085953,3.132439,4.491739,3.354165,3.176818,3.280135,3.354467,4.792292,3.511842,...,3.021057,3.175435,3.263664,4.828807,4.465866,2.672297,3.412282,6.316696,5.806777,3.881669
4,7.740312,6.932358,8.302799,9.026284,10.569610,7.795554,7.587172,8.664989,8.446903,8.637650,...,6.622842,7.470787,7.503838,7.774821,7.381567,6.479749,8.017678,6.438013,6.345137,7.368461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,2.183641,2.868645,2.215248,2.928358,2.033018,3.129356,2.199877,2.524856,4.061677,2.264357,...,5.166986,6.002409,6.856323,6.476348,5.100714,6.117684,6.888410,4.181026,2.529937,0.922690
10860,3.860065,6.245387,8.674895,8.582032,7.174702,4.459911,6.065849,8.514661,9.126066,8.038447,...,4.927103,5.414635,5.832173,7.091899,6.286269,5.857142,4.894269,4.369744,5.375127,5.760761
10861,9.422306,9.833978,8.797062,8.938500,10.587920,11.111170,10.294480,10.483610,10.621140,10.243910,...,10.388580,10.736290,12.012380,11.541850,10.367420,10.303140,12.195170,11.727460,12.455130,12.188480
10862,10.168480,10.181580,9.827646,11.009760,11.220900,9.853840,10.186380,10.157660,10.248250,10.999120,...,8.738751,8.591082,8.628104,8.130602,9.341041,6.525043,7.262946,7.761230,7.102003,6.825168


In [16]:
placurv_cols = [col for col in train.columns if col.endswith('placurv')]
train[placurv_cols]

,1_placurv,2_placurv,3_placurv,4_placurv,5_placurv,6_placurv,7_placurv,8_placurv,9_placurv,10_placurv,...,16_placurv,17_placurv,18_placurv,19_placurv,20_placurv,21_placurv,22_placurv,23_placurv,24_placurv,25_placurv
0,0.038514,0.029463,0.031405,0.025771,0.010453,0.028321,0.027360,0.019831,0.009338,0.008806,...,0.023851,0.012586,-0.004249,0.003646,-0.006809,0.013328,0.005337,0.007039,-0.017753,-0.007777
1,0.007030,0.012162,0.017932,0.010745,0.017279,0.005049,0.010853,0.013505,0.022271,0.017100,...,0.002180,0.008424,0.000325,-0.002641,0.002723,0.019174,0.010954,0.005607,0.005583,-0.000081
2,0.007581,0.016033,0.032177,0.026111,0.019280,0.027318,0.022358,0.038340,0.028284,0.013423,...,0.060813,0.050363,0.042203,0.011481,-0.002997,0.081775,0.057673,0.035016,0.010485,-0.012906
3,0.033529,0.030153,0.032375,0.041220,0.038542,0.035620,0.038546,0.036121,0.033105,0.035626,...,0.026844,0.028277,0.022927,0.021857,0.011739,0.023054,0.021450,0.019539,0.017652,0.017803
4,0.005615,-0.002092,-0.011539,-0.011659,-0.009951,0.015373,-0.002021,0.000540,-0.011140,-0.006108,...,-0.000948,-0.002362,-0.002638,-0.006659,-0.014926,0.007847,-0.005283,-0.015737,-0.003549,-0.018709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,0.016162,0.032614,0.031390,0.030518,0.026860,0.013465,0.015114,0.018182,0.025875,0.017596,...,0.009773,-0.000059,0.002935,-0.000134,0.008460,0.001812,-0.000082,0.003379,-0.001462,-0.007938
10860,0.032583,0.016913,0.009984,0.005113,-0.009874,0.039336,0.015624,0.009280,0.007921,-0.002632,...,0.036308,0.037973,0.030251,0.019123,0.011850,0.022944,0.023571,0.029078,0.028585,0.020360
10861,0.023251,0.033821,0.021196,0.030915,0.021522,0.009237,0.006101,0.021416,0.020860,0.018128,...,-0.008116,-0.010156,-0.014071,-0.002608,0.003364,-0.035174,-0.025445,-0.027863,-0.029843,-0.024226
10862,0.008236,-0.005702,-0.026834,-0.032061,-0.004075,0.010196,-0.014070,-0.023399,-0.017773,-0.000540,...,-0.003354,-0.012173,0.001411,0.006976,0.026990,-0.004198,0.004627,0.008300,0.027105,0.038743


In [17]:
procurv_cols = [col for col in train.columns if col.endswith('procurv')]
train[procurv_cols]

,1_procurv,2_procurv,3_procurv,4_procurv,5_procurv,6_procurv,7_procurv,8_procurv,9_procurv,10_procurv,...,16_procurv,17_procurv,18_procurv,19_procurv,20_procurv,21_procurv,22_procurv,23_procurv,24_procurv,25_procurv
0,0.013903,0.014674,0.018249,0.018368,0.003341,0.002023,0.005743,0.002238,-0.003820,-0.003302,...,0.000977,-0.012630,-0.031644,-0.025683,-0.037371,-0.018934,-0.021875,-0.023553,-0.048495,-0.039092
1,-0.012546,-0.006645,-0.001383,-0.013503,-0.006243,-0.010565,-0.008093,-0.005231,-0.000201,-0.008825,...,0.000579,0.005370,-0.005845,-0.005632,-0.005483,0.019445,0.008355,-0.002847,-0.002825,-0.008197
2,-0.015857,0.000517,0.009202,0.009751,0.002789,0.005787,0.007985,0.014076,0.010337,0.003131,...,0.046773,0.048948,0.037799,0.021622,0.022307,0.080986,0.060944,0.044985,0.041931,0.023939
3,0.018884,0.011227,0.009005,0.008436,0.011113,0.019552,0.011109,0.008016,-0.008278,0.005753,...,0.020053,0.015860,0.007418,0.000211,-0.000705,0.021083,0.014413,0.008047,0.001658,0.001507
4,-0.000098,-0.003424,-0.010530,-0.007653,-0.006601,0.009456,-0.000739,0.002218,-0.005411,-0.002167,...,-0.012844,-0.008673,-0.005639,-0.009892,-0.015465,-0.007849,-0.014026,-0.020124,-0.010269,-0.022651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,0.016938,0.028078,0.018263,0.010769,0.000504,0.016882,0.012287,0.011788,0.012556,-0.001267,...,-0.007106,-0.016658,-0.017000,-0.016208,-0.008337,-0.018422,-0.019210,-0.014214,-0.020589,-0.028007
10860,0.030867,0.018946,0.009327,0.011439,-0.001164,0.026873,0.014725,0.010029,0.011391,0.002629,...,0.021621,0.019957,0.011131,0.008460,0.007461,0.012915,0.004017,0.006784,0.012797,0.004467
10861,0.029166,0.032382,0.011902,0.021499,0.003304,0.015592,0.007695,0.017210,0.012243,0.003945,...,0.002601,-0.003635,-0.010756,-0.002907,-0.003366,-0.014481,-0.007658,-0.013517,-0.014300,-0.017149
10862,0.022108,0.011219,-0.006269,-0.014837,-0.004200,0.025667,0.014070,0.006849,0.003979,0.000539,...,0.019906,0.017690,0.023416,0.020612,0.022666,0.006958,0.020200,0.016528,0.028068,0.030222


In [18]:
sdoif_cols = [col for col in train.columns if col.endswith('sdoif')]
train[sdoif_cols]

,1_sdoif,2_sdoif,3_sdoif,4_sdoif,5_sdoif,6_sdoif,7_sdoif,8_sdoif,9_sdoif,10_sdoif,...,16_sdoif,17_sdoif,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif
0,1.281767,1.281743,1.281708,1.281684,1.281649,1.281780,1.281757,1.281721,1.281698,1.281662,...,1.281802,1.281779,1.281743,1.281720,1.281684,1.281811,1.281788,1.281752,1.281729,1.281693
1,1.359568,1.359547,1.359516,1.359495,1.359463,1.359602,1.359582,1.359550,1.359530,1.359498,...,1.359660,1.359639,1.359608,1.359587,1.359556,1.359683,1.359662,1.359631,1.359610,1.359579
2,1.365062,1.365091,1.365111,1.365141,1.365161,1.365037,1.365067,1.365087,1.365117,1.365137,...,1.364975,1.365005,1.365025,1.365055,1.365075,1.364937,1.364967,1.364988,1.365018,1.365038
3,1.100921,1.100941,1.100971,1.100992,1.101023,1.100834,1.100854,1.100884,1.100904,1.100935,...,1.100688,1.100708,1.100738,1.100759,1.100789,1.100630,1.100650,1.100680,1.100700,1.100731
4,1.284558,1.284483,1.284433,1.284358,1.284308,1.284471,1.284397,1.284347,1.284272,1.284222,...,1.284255,1.284180,1.284130,1.284056,1.284006,1.284125,1.284050,1.284001,1.283926,1.283876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,1.272353,1.272419,1.272516,1.272582,1.272679,1.272163,1.272229,1.272327,1.272392,1.272490,...,1.271847,1.271912,1.272011,1.272076,1.272174,1.271720,1.271786,1.271884,1.271950,1.272048
10860,1.331059,1.331035,1.331019,1.330994,1.330978,1.331104,1.331079,1.331063,1.331039,1.331023,...,1.331178,1.331153,1.331137,1.331113,1.331096,1.331207,1.331183,1.331167,1.331142,1.331126
10861,1.298319,1.298303,1.298279,1.298263,1.298239,1.298341,1.298325,1.298301,1.298285,1.298260,...,1.298378,1.298362,1.298337,1.298321,1.298297,1.298392,1.298376,1.298352,1.298336,1.298311
10862,1.355228,1.355267,1.355293,1.355332,1.355357,1.355141,1.355180,1.355206,1.355245,1.355271,...,1.354996,1.355035,1.355061,1.355099,1.355125,1.354937,1.354976,1.355002,1.355041,1.355067


In [19]:
slope_cols = [col for col in train.columns if col.endswith('slope')]
train[slope_cols]

,1_slope,2_slope,3_slope,4_slope,5_slope,6_slope,7_slope,8_slope,9_slope,10_slope,...,16_slope,17_slope,18_slope,19_slope,20_slope,21_slope,22_slope,23_slope,24_slope,25_slope
0,35.26439,37.29208,33.85452,35.79576,40.31554,38.87666,39.50971,40.51059,45.83452,45.00000,...,33.85452,38.87666,33.85452,33.85452,32.63194,32.63194,30.24626,30.24626,30.24626,30.24626
1,32.31153,32.31153,26.56505,21.80141,22.40687,27.01712,35.26439,31.31116,21.80141,21.80141,...,26.56505,27.01712,35.26439,31.31116,27.01712,26.56505,26.56505,30.96376,34.99202,31.31116
2,42.67464,43.33172,40.70319,35.79576,32.31153,56.77032,53.67613,46.23402,40.51059,36.05503,...,45.00000,36.05503,35.26439,31.31116,21.80141,37.99073,27.01712,27.01712,28.30320,26.56505
3,19.82703,17.54840,17.54840,22.40687,16.69924,16.69924,16.69924,16.69924,21.80141,16.69924,...,17.54840,16.69924,17.54840,24.09484,22.40687,15.79317,19.82703,30.24626,26.56505,19.82703
4,32.63194,26.56505,30.24626,29.49621,30.24626,32.63194,29.49621,30.24626,30.24626,26.56505,...,26.56505,30.24626,29.49621,26.56505,24.09484,26.56505,30.24626,26.56505,22.98977,24.09484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,12.60438,15.79317,12.60438,15.79317,12.60438,16.69924,12.60438,15.79317,19.82703,12.60438,...,21.80141,28.30320,30.24626,28.30320,27.01712,24.09484,28.30320,19.82703,19.82703,17.54840
10860,19.82703,28.30320,36.05503,33.85452,29.49621,22.98977,28.30320,36.05503,36.05503,30.24626,...,24.09484,26.56505,28.30320,30.96376,26.56505,27.01712,24.09484,22.98977,27.01712,26.56505
10861,37.99073,40.31554,37.99073,38.87666,43.33172,40.70319,40.70319,42.67464,44.71062,42.67464,...,35.26439,37.99073,40.70319,40.31554,37.99073,35.79576,40.31554,40.70319,40.70319,40.31554
10862,37.99073,33.85452,33.85452,38.87666,39.50971,35.26439,33.85452,36.05503,39.50971,42.67464,...,31.31116,35.26439,35.26439,35.26439,39.50971,26.56505,30.96376,35.26439,31.31116,32.31153


In [20]:
twi_cols = [col for col in train.columns if col.endswith('twi')]
train[twi_cols]

,1_twi,2_twi,3_twi,4_twi,5_twi,6_twi,7_twi,8_twi,9_twi,10_twi,...,16_twi,17_twi,18_twi,19_twi,20_twi,21_twi,22_twi,23_twi,24_twi,25_twi
0,3.173340,2.961406,3.315935,3.061250,3.246914,3.221661,3.153512,3.048637,2.915506,3.073973,...,3.851345,3.673898,3.821337,3.584646,3.734637,4.003083,4.218082,3.990867,4.100921,3.715154
1,4.396348,4.315768,4.722144,5.003747,4.654147,4.379892,4.109311,4.659479,5.148889,5.133717,...,4.642571,4.441799,4.201243,4.646032,4.661026,4.732493,4.869680,4.789931,4.580217,4.984037
2,3.912020,4.522530,4.679744,4.842173,4.636874,2.708843,3.363745,4.501287,4.726338,4.938828,...,2.838589,3.283489,3.966261,5.133079,5.904688,3.026198,3.733081,4.182730,4.977861,5.951684
3,3.021531,3.409663,3.484420,3.497207,4.191795,3.920182,4.080204,4.192245,4.020496,4.421484,...,3.303394,3.918005,3.689611,3.331842,3.468323,3.467204,3.015849,3.043601,3.549856,3.660269
4,3.523277,4.435728,4.410583,5.006793,5.617533,3.558835,4.138383,4.624073,4.496619,5.535381,...,4.207351,3.882622,4.083162,5.009180,5.453774,4.098137,4.235864,4.065827,5.037767,5.444889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,4.128441,3.821712,4.200294,3.924722,3.771079,3.844918,4.165479,3.183432,3.886923,4.309926,...,4.396901,3.261328,3.453476,3.677230,4.185551,4.514762,3.983744,4.119469,5.008305,5.979714
10860,3.632363,3.459739,3.395988,3.781246,3.858895,3.274968,3.313896,3.302768,3.649495,4.248799,...,3.432600,3.200255,3.117472,3.456191,3.946566,3.471856,3.399169,3.172846,3.042458,3.510076
10861,3.447679,3.254569,3.104369,3.025702,3.135352,3.799932,3.418217,3.188772,2.940966,3.073126,...,4.451507,4.100429,4.189867,4.055251,3.925623,4.306184,4.330550,4.069845,4.370842,4.327808
10862,3.828743,4.635796,4.458890,4.067776,4.052110,4.187277,4.638149,4.184963,3.598754,3.428784,...,4.421325,3.501594,3.523095,3.226146,3.135310,4.132965,3.575352,2.993675,3.384373,2.963617


In [21]:
aspect_cols = [col for col in train.columns if col.endswith('aspect')]
train[aspect_cols]

,1_aspect,2_aspect,3_aspect,4_aspect,5_aspect,6_aspect,7_aspect,8_aspect,9_aspect,10_aspect,...,16_aspect,17_aspect,18_aspect,19_aspect,20_aspect,21_aspect,22_aspect,23_aspect,24_aspect,25_aspect
0,98.1301,113.1986,116.5650,123.6901,135.0000,97.12502,104.0362,110.5560,119.0546,126.8699,...,116.5650,119.7449,116.5650,116.5650,128.6598,128.6598,120.9638,120.9638,120.9638,120.9638
1,198.4350,198.4350,180.0000,180.0000,194.0362,191.30990,188.1301,189.4623,180.0000,180.0000,...,180.0000,191.3099,188.1301,189.4623,168.6901,180.0000,180.0000,180.0000,180.0000,170.5377
2,310.6013,302.0054,305.5377,303.6901,288.4349,301.60750,287.1027,286.6992,290.5560,285.9454,...,306.8699,285.9454,261.8699,260.5377,270.0000,309.8056,281.3099,258.6901,248.1986,270.0000
3,213.6901,198.4350,198.4350,194.0362,180.0000,180.00000,180.0000,180.0000,180.0000,180.0000,...,161.5650,180.0000,161.5650,153.4350,165.9638,135.0000,146.3099,149.0362,143.1301,146.3099
4,128.6598,126.8699,120.9638,135.0000,120.9638,128.65980,135.0000,120.9638,120.9638,126.8699,...,126.8699,120.9638,135.0000,126.8699,116.5650,126.8699,120.9638,126.8699,135.0000,116.5650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,116.5650,135.0000,153.4350,135.0000,153.4350,90.00000,116.5650,135.0000,146.3099,116.5650,...,90.0000,111.8014,120.9638,111.8014,101.3099,116.5650,111.8014,123.6901,123.6901,108.4350
10860,146.3099,158.1986,164.0546,153.4350,135.0000,135.00000,158.1986,164.0546,164.0546,149.0362,...,116.5650,126.8699,158.1986,180.0000,180.0000,101.3099,116.5650,135.0000,168.6901,180.0000
10861,129.8056,135.0000,140.1944,150.2551,147.9946,125.53770,125.5377,130.6013,135.0000,139.3987,...,135.0000,140.1944,144.4623,135.0000,129.8056,146.3099,135.0000,144.4623,144.4623,135.0000
10862,219.8056,243.4350,243.4350,240.2551,255.9638,225.00000,243.4350,254.0546,255.9638,257.4712,...,260.5377,261.8699,261.8699,261.8699,255.9638,270.0000,270.0000,261.8699,260.5377,251.5650


In [25]:
geology_cols = [col for col in train.columns if col.endswith('geology')]
train[geology_cols]

,1_geology,2_geology,3_geology,4_geology,5_geology,6_geology,7_geology,8_geology,9_geology,10_geology,...,16_geology,17_geology,18_geology,19_geology,20_geology,21_geology,22_geology,23_geology,24_geology,25_geology
0,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
4,2,2,2,5,5,2,2,5,5,5,...,2,2,5,5,5,2,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,2,2,2,5,5,2,2,2,2,5,...,2,2,2,2,2,2,2,2,2,2
10860,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
10861,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
10862,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [26]:
data = pd.concat([train.assign(train=1), test.assign(train=0)])
print(data.shape)
data.head(5)

(16294, 228)


,Sample_ID,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,...,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif,Label,train
0,1,130,129,127,126,123,126,125,124,122,...,1.281743,1.281720,1.281684,1.281811,1.281788,1.281752,1.281729,1.281693,0.0,1
1,2,161,158,155,153,151,162,159,155,153,...,1.359608,1.359587,1.359556,1.359683,1.359662,1.359631,1.359610,1.359579,1.0,1
2,3,149,151,154,156,158,154,157,158,160,...,1.365025,1.365055,1.365075,1.364937,1.364967,1.364988,1.365018,1.365038,0.0,1
3,4,80,78,77,75,73,80,78,77,75,...,1.100738,1.100759,1.100789,1.100630,1.100650,1.100680,1.100700,1.100731,0.0,1
4,5,117,115,114,112,110,115,113,111,110,...,1.284130,1.284056,1.284006,1.284125,1.284050,1.284001,1.283926,1.283876,0.0,1


In [27]:

base_fts = ['elevation', 'lsfactor', 'placurv', 'procurv', 'sdoif', 'slope', 'twi', 'aspect']

df = pd.DataFrame()
for i in base_fts:
    cols = [x for x in data.columns if x.endswith(i)]
    df[f'{i}_mean'] = data[cols].mean(axis=1)
    df[f'{i}_max'] = data[cols].max(axis=1)
    df[f'{i}_min'] = data[cols].min(axis=1)
    df[f'{i}_std'] = data[cols].std(axis=1)
    df[f'{i}_skew'] = data[cols].skew(axis=1)
    df[f'{i}_sum'] = data[cols].sum(axis=1)
    df[f'{i}_sem'] = data[cols].sem(axis=1)
    df[f'{i}_kurtosis'] = data[cols].kurtosis(axis=1)

for i in ['geology']:
    cols = [x for x in data.columns if x.endswith(i)]
    df[f'{i}_count'] = data[cols].count(axis=1)
    df[f'{i}_quantile'] = data[cols].quantile(axis=1)
    df[f'{i}_mean'] = data[cols].mean(axis=1)
    
print(df.shape)
df.head()

(16294, 67)


,elevation_mean,elevation_max,elevation_min,elevation_std,elevation_skew,elevation_sum,elevation_sem,elevation_kurtosis,lsfactor_mean,lsfactor_max,...,aspect_max,aspect_min,aspect_std,aspect_skew,aspect_sum,aspect_sem,aspect_kurtosis,geology_count,geology_quantile,geology_mean
0,119.44,130,110,5.795688,0.164882,2986,1.159138,-1.046021,9.013694,11.035840,...,135.0000,97.12502,9.057176,-0.448383,2921.61402,1.811435,0.403625,25,3.0,3.0
1,156.20,162,150,4.092676,0.069766,3905,0.818535,-1.419892,8.013825,10.544930,...,198.4350,168.69010,8.046540,0.146889,4615.83590,1.609308,-0.450431,25,3.0,3.0
2,162.56,171,149,6.474823,-0.517681,4064,1.294965,-0.770406,10.958018,15.173530,...,310.6013,248.19860,17.485790,-0.284634,7123.56090,3.497158,-0.748418,25,2.0,2.0
3,76.40,80,72,2.565801,-0.180178,1910,0.513160,-1.180694,3.785720,6.316696,...,213.6901,135.00000,19.374545,-0.202386,4326.91120,3.874909,-0.291926,25,2.0,2.0
4,109.16,117,102,3.954744,0.103775,2729,0.790949,-0.647523,7.742521,10.569610,...,135.0000,116.56500,6.084374,0.291106,3137.81430,1.216875,-0.939511,25,5.0,3.8


In [28]:
train_df = data[data.train == 1].drop(columns=['train'])
test_df = data[data.train == 0].drop(columns=['train', 'Label'])

train_df.shape, test_df.shape

((10864, 227), (5430, 226))

In [42]:
import time 
from sklearn.metrics import f1_score,mean_squared_error

def squared_mean(results):
    mean_results = np.ones(results[0].shape)
    ln = len(results)
    for i in range(ln):
        mean_results *= results[i]

    return mean_results ** (1/ln)
class My_model:
    def __init__(self, models):
        self.models = models

    def fit_eval_pred(self, X, y, eval_set, val, test):
        results_eval = []
        results_test = []
        for i, model in enumerate(self.models):
            print(f"[Training]............................... Model_{i+1}")
            st = time.time()
            try:
                model.fit(X, y, eval_set=eval_set, verbose=False)
            except Exception as e:
                model.fit(X, y)
            print(f"[Prediction]............................. Model_{i+1}")
            p = model.predict_proba(val)
            print(f"Log loss = {f1_score(eval_set[0][1], np.argmax(p, axis=1))}", end=" "*6)
            results_eval.append(p)
            results_test.append(model.predict_proba(test))
            print(f"Time {time.time() - st :.2f}sec")

        results_eval = squared_mean(results_eval)
        results_test = squared_mean(results_test)

        return results_eval, results_test

In [45]:
import time
from sklearn.model_selection import StratifiedKFold
#from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import imblearn.over_sampling as ovs


seed = 21

def get_models(seed=21):

    cat_params = {"iterations":10000, "learning_rate":0.1, "verbose":200, "random_state":seed,
            'use_best_model':True, 'early_stopping_rounds':200, }

    lgb_params = {'n_estimators':300, 'random_state':seed, 'early_stopping_round':100, 'verbose':-1, 
            'learning_rate': 0.06918642648131483, 'num_leaves': 20, 'max_depth': 11,
            'min_data_in_leaf': 200, 'max_bin': 200, 'lambda_l1': 10, 'lambda_l2': 20,
            'min_gain_to_split': 0.00011324914950008869, 'bagging_fraction': 0.6000000000000001,
            'bagging_freq': 1, 'feature_fraction': 0.8
            }
            
    xgb_params={'learning_rate': 0.23888500155357034,'booster': 'dart','reg_lambda': 0.34664605600334886 ,'subsample': 0.8075258986325098,
     'reg_alpha': 0.71641157762337,'max_depth': 6,'min_child_weight': 2.6364948620856137e-08,'random_state': 24,'n_estimators': 226}

    return  [
            XGBClassifier(**xgb_params),   
            LGBMClassifier(**lgb_params),
    ]

In [46]:
X = train_df.drop('Label',axis=1)
y = train_df.Label

results = []
ns = 10
tot = 0

skf = StratifiedKFold(n_splits=ns, random_state=seed, shuffle=True)
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(100*"-", f"\nFold-{fold+1}")

    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    sampler = ovs.SMOTE(random_state=42)
    X_train, y_train = sampler.fit_resample(X_train, y_train)

    main_model = My_model(get_models())
    pred, T_pred = main_model.fit_eval_pred(X_train, y_train, [(X_test, y_test)], X_test, test)

    score = f1_score(y_test, pd.Series([[0, 1][i >= .35] for i in pred[:, 1]]))

    print(f"f1 score = {score}")
    tot += score

    results.append(T_pred)

    print(100*"-", "\n\n\n")

print(f"Average log loss = {tot / ns}")

---------------------------------------------------------------------------------------------------- 
Fold-1
[Training]............................... Model_1
[Prediction]............................. Model_1
Log loss = 0.6953271028037383      Time 262.42sec
[Training]............................... Model_2
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l1 is set=10, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10
[LightGBM] [Warning] bagging_fraction is set=0.6000000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6000000000000001
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=0.00011324914950008869, min_split_gain=0.

In [52]:
results

[array([[0.29621065, 0.6612051 ],
        [0.97788412, 0.00422776],
        [0.9806035 , 0.01621821],
        ...,
        [0.97368081, 0.008065  ],
        [0.98416052, 0.00403271],
        [0.45293791, 0.50708691]]),
 array([[0.30392705, 0.69363495],
        [0.98118103, 0.01603043],
        [0.99136811, 0.00292684],
        ...,
        [0.96282972, 0.0139956 ],
        [0.98465126, 0.00566886],
        [0.32041423, 0.67886963]]),
 array([[0.07496495, 0.84385372],
        [0.97040829, 0.01207153],
        [0.98131532, 0.0140112 ],
        ...,
        [0.9691076 , 0.01644638],
        [0.97907693, 0.00630282],
        [0.26960772, 0.73029221]]),
 array([[0.08090186, 0.87236864],
        [0.9878389 , 0.00634632],
        [0.98884241, 0.00222751],
        ...,
        [0.97530168, 0.00857393],
        [0.98964998, 0.00303646],
        [0.32845622, 0.6706315 ]]),
 array([[0.35000606, 0.52540527],
        [0.98825894, 0.00175746],
        [0.98252302, 0.00305903],
        ...,
        [

In [65]:
test_pred = [[0, 1][i >= .35] for i in T_pred[:, 1]]
test_pred

[1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,


In [63]:
ss.head()

,Sample_ID,Label
0,10865,0
1,10866,0
2,10867,0
3,10868,0
4,10869,0


In [66]:
ss.Label = test_pred

In [ ]:
ss.to_csv('submission.csv')